# Q1. Prepare documents

In [1]:
import json

with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

len(documents)

948

Q1 --> 948

# Q2

In [2]:
# This is a new library compared to the previous modules. 
# Please perform "pip install sentence_transformers==2.7.0"

from sentence_transformers import SentenceTransformer

# if you get an error do the following:
# 1. Uninstall numpy 
# 2. Uninstall torch
# 3. pip install numpy==1.26.4
# 4. pip install torch
# run the above cell, it should work
model = SentenceTransformer("all-MiniLM-L12-v2")

/Users/kamilakolpashnikova/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Q2 --> [120, 59.76]

# Q3

In [3]:
len(model.encode("This is a simple sentence"))

384

Q3 --> 384

In [4]:
#created the dense vector using the pre-trained model
operations = []
for doc in documents:
    # Transforming the title into an embedding using the model
    doc["question_vector"] = model.encode(doc["question"]).tolist()
    operations.append(doc)

In [6]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': '5edc26a08bfd', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'vsVMeOj8SBi_gvJy0gH-kQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

# Q4

In [7]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "keyword"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "question_vector":{"type":"dense_vector","dims": 384,"index":True,"similarity": "cosine"
        },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [8]:
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

Q4 --> If the index is unavailable, skip this statement

# Q5

In [9]:
search_term = "how to enrol to course?"
vector_search_term = model.encode(search_term)

query = {
    "field" : "question_vector",
    "query_vector" :  vector_search_term,
    "k" : 5,
    "num_candidates" : 10000, 
}

res = es_client.search(index=index_name, knn=query,source=["text","section","question","course"])
res["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': '8aMQjpABucWgZAQQ_Brf',
  '_score': 0.85204816,
  '_source': {'question': 'Course - What are the prerequisites for this course?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites'}},
 {'_index': 'course-questions',
  '_id': 'EqMQjpABucWgZAQQ_Rty',
  '_score': 0.83816737,
  '_source': {'question': 'How can we contribute to the course?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.'}},
 {'_index': 'course-questions',
  '_id': 'tKMRjpABucWgZAQQAxxN',
  '_score': 0.828423,
  '_source': {'question': 'I just joined. What should I do next? How can I access course materials?',
   'course': 'machine-learning-zoomcamp',
   'section': 'Ge

Q5 --> [0.74, 'General course-related questions', 'data-engineering-zoomcamp']

# Q6

In [10]:
knn_query = {
    "field" : "text_vector",
    "query_vector" :  vector_search_term,
    "k" : 5,
    "num_candidates" : 10000
}

response = es_client.search(
    index=index_name,
    query={
        "match": {
                "section": "General course-related questions"
            },
        },
        
    knn=knn_query,
    size=5
)

response["hits"]["hits"]

BadRequestError: BadRequestError(400, 'search_phase_execution_exception', 'failed to create query: field [text_vector] does not exist in the mapping')

Q6 --> No returns an error --> BadRequestError(400, 'search_phase_execution_exception', 'failed to create query: field [text_vector] does not exist in the mapping') --> we didn't encode it

In [12]:
## this one is the fixed with the correct field name

knn_query = {
    "field" : "question_vector",
    "query_vector" :  vector_search_term,
    "k" : 5,
    "num_candidates" : 10000
}

response = es_client.search(
    index=index_name,
    query={
        "match": {
                "section": "General course-related questions"
            },
        },
        
    knn=knn_query,
    size=5
)

response["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': '8aMQjpABucWgZAQQ_Brf',
  '_score': 3.4953294,
  '_source': {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
   'section': 'General course-related questions',
   'question': 'Course - What are the prerequisites for this course?',
   'course': 'data-engineering-zoomcamp',
   'question_vector': [0.04312112554907799,
    0.058370333164930344,
    0.005022970959544182,
    0.02857058495283127,
    -0.022065436467528343,
    0.004731933120638132,
    -0.025459865108132362,
    -0.0008887263829819858,
    -0.16603535413742065,
    -0.004694811068475246,
    -0.05639376491308212,
    -0.10678310692310333,
    -0.06021315976977348,
    0.039712488651275635,
    -0.06361556798219681,
    0.03923671320080757,
    -0.004405471961945295,
    -0.008392639458179474,
    0.14483696222305298,
    -0.07729385048151016,
    0.015076606534421444,
    0.02086200751364231,
    0.01940774917602539,
    -0.03106939233839512,
    0.00793197378